<hr style="border:0.1px solid gray">

# <center style='color:Red'> COSC2753 - Machine Learning </center> 
## <center style='color:Red'> Assignment 2 - Classify Images of Colon Cancer </center>
### <center style='color:Red'> Task 3 - Classify extra data cell type </center>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#-COSC2753---Machine-Learning-" data-toc-modified-id="-COSC2753---Machine-Learning--1"><center style="color:Red"> COSC2753 - Machine Learning </center></a></span><ul class="toc-item"><li><span><a href="#-Assignment-2---Classify-Images-of-Colon-Cancer-" data-toc-modified-id="-Assignment-2---Classify-Images-of-Colon-Cancer--1.1"><center style="color:Red"> Assignment 2 - Classify Images of Colon Cancer </center></a></span><ul class="toc-item"><li><span><a href="#-Task-3---Classify-extra-data-cell-type-" data-toc-modified-id="-Task-3---Classify-extra-data-cell-type--1.1.1"><center style="color:Red"> Task 3 - Classify extra data cell type </center></a></span></li></ul></li></ul></li><li><span><a href="#Main-Task" data-toc-modified-id="Main-Task-2">Main Task</a></span></li><li><span><a href="#1.-Import-libraries-and-setting-up-environment" data-toc-modified-id="1.-Import-libraries-and-setting-up-environment-3">1. Import libraries and setting up environment</a></span></li><li><span><a href="#2.-Import-Dataset" data-toc-modified-id="2.-Import-Dataset-4">2. Import Dataset</a></span></li><li><span><a href="#3.-Exploratory-Data-Analysis-(EDA)" data-toc-modified-id="3.-Exploratory-Data-Analysis-(EDA)-5">3. Exploratory Data Analysis (EDA)</a></span><ul class="toc-item"><li><span><a href="#3.1.-Process-the-data" data-toc-modified-id="3.1.-Process-the-data-5.1">3.1. Process the data</a></span></li><li><span><a href="#3.2.-Data-Analysis-and-Visualiztion" data-toc-modified-id="3.2.-Data-Analysis-and-Visualiztion-5.2">3.2. Data Analysis and Visualiztion</a></span></li><li><span><a href="#3.3.-Data-Preparation" data-toc-modified-id="3.3.-Data-Preparation-5.3">3.3. Data Preparation</a></span></li></ul></li><li><span><a href="#4.-Building-GAN-model" data-toc-modified-id="4.-Building-GAN-model-6">4. Building GAN model</a></span><ul class="toc-item"><li><span><a href="#4.1-Function-define" data-toc-modified-id="4.1-Function-define-6.1">4.1 Function define</a></span></li><li><span><a href="#------------------------" data-toc-modified-id="-------------------------6.2">------------------------</a></span><ul class="toc-item"><li><span><a href="#Image-functions" data-toc-modified-id="Image-functions-6.2.1">Image functions</a></span></li></ul></li><li><span><a href="#------------------------" data-toc-modified-id="-------------------------6.3">------------------------</a></span><ul class="toc-item"><li><span><a href="#Generate-samples" data-toc-modified-id="Generate-samples-6.3.1">Generate samples</a></span></li></ul></li><li><span><a href="#------------------------" data-toc-modified-id="-------------------------6.4">------------------------</a></span><ul class="toc-item"><li><span><a href="#Generator" data-toc-modified-id="Generator-6.4.1">Generator</a></span></li></ul></li><li><span><a href="#------------------------" data-toc-modified-id="-------------------------6.5">------------------------</a></span><ul class="toc-item"><li><span><a href="#Discriminator" data-toc-modified-id="Discriminator-6.5.1">Discriminator</a></span></li></ul></li><li><span><a href="#------------------------" data-toc-modified-id="-------------------------6.6">------------------------</a></span><ul class="toc-item"><li><span><a href="#GAN" data-toc-modified-id="GAN-6.6.1">GAN</a></span></li></ul></li><li><span><a href="#------------------------" data-toc-modified-id="-------------------------6.7">------------------------</a></span></li><li><span><a href="#4.2-Model-training" data-toc-modified-id="4.2-Model-training-6.8">4.2 Model training</a></span></li><li><span><a href="#4.3-Prediction" data-toc-modified-id="4.3-Prediction-6.9">4.3 Prediction</a></span></li></ul></li></ul></div>

**Group Number:** 8

**Student 1:** Ngo Quang Khai - s3836387 

**Student 2:** Nguyen Quoc Minh - s3758994

**Student 3:** Bui Thanh Huy - s3740934

**Student 4:** Tran Vinh Khang - s3855823

**Student 5:** Le Trong Hung - s3805504

**Lecturer:** Dr. Bao Nguyen

**Due Date:** May 14th, 2021

<hr style="border:0.1px solid gray">

# Main Task

- **Task**: Classify cell type on extra data

# 1. Import libraries and setting up environment

In [53]:
# !pip install tensorflow
# !pip install keras
# !pip install opencv-python
# !pip install bayesian-optimization 

In [54]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn as sk
import seaborn as sns
import cv2
import os
import tensorflow as tf

from numpy import expand_dims, zeros, ones, asarray
from numpy.random import randn, randint
from tqdm import tqdm
from sklearn.model_selection import train_test_split, cross_val_score
from imblearn.over_sampling import RandomOverSampler

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import layers
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.regularizers import l2
from keras import backend as K
from math import floor
from sklearn.metrics import make_scorer, accuracy_score, classification_report

from sklearn.model_selection import StratifiedKFold,GridSearchCV

from PIL import Image
import warnings
warnings.filterwarnings('ignore')

LeakyReLU = LeakyReLU(alpha=0.1)

In [55]:
#It is important to check the version of these libraries to make sure that implement the latest method.
print('Tensorflow version:', tf.__version__)
print('CV2 version:', cv2.__version__)

Tensorflow version: 2.6.0
CV2 version: 4.5.5


# 2. Import Dataset

In [56]:
# Import data_labels_mainData.csv into a DataFrame
main_data = pd.read_csv('data_labels_mainData.csv')

# Import data_labels_extraData.csv into a DataFrame
extra_data = pd.read_csv('data_labels_extraData.csv')

# 3. Exploratory Data Analysis (EDA)

## 3.1. Process the data

In [57]:
#See the data frame of main_data
print("Dataframe of main_data: \n", main_data, "\n")

#See the data frame of extra_data
print("Dataframe of extra_data: \n", extra_data)

Dataframe of main_data: 
       InstanceID  patientID  ImageName cellTypeName  cellType  isCancerous
0          22405          1  22405.png   fibroblast         0            0
1          22406          1  22406.png   fibroblast         0            0
2          22407          1  22407.png   fibroblast         0            0
3          22408          1  22408.png   fibroblast         0            0
4          22409          1  22409.png   fibroblast         0            0
...          ...        ...        ...          ...       ...          ...
9891        1625         60   1625.png   epithelial         2            1
9892        1626         60   1626.png   epithelial         2            1
9893        1627         60   1627.png   epithelial         2            1
9894        1628         60   1628.png   epithelial         2            1
9895        1629         60   1629.png   epithelial         2            1

[9896 rows x 6 columns] 

Dataframe of extra_data: 
        InstanceID  p

**=>** Next, we will check if the data have any null values and datatype of each columns.

In [58]:
main_data.info()
main_data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9896 entries, 0 to 9895
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   InstanceID    9896 non-null   int64 
 1   patientID     9896 non-null   int64 
 2   ImageName     9896 non-null   object
 3   cellTypeName  9896 non-null   object
 4   cellType      9896 non-null   int64 
 5   isCancerous   9896 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 464.0+ KB


InstanceID      0
patientID       0
ImageName       0
cellTypeName    0
cellType        0
isCancerous     0
dtype: int64

In [59]:
extra_data.info()
extra_data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10384 entries, 0 to 10383
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   InstanceID   10384 non-null  int64 
 1   patientID    10384 non-null  int64 
 2   ImageName    10384 non-null  object
 3   isCancerous  10384 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 324.6+ KB


InstanceID     0
patientID      0
ImageName      0
isCancerous    0
dtype: int64

**=>** Since our data is no missing/null value and properly formatted, we will move on to analyzing data.

## 3.2. Data Analysis and Visualiztion

Based on the information provided by the biomedical company as well as the DataFrames in **section 3.1** above. We are aware of:
- The **main_data** contains both **cellType (0 to 3)** and **isCancerous (0 and 1)** columns. In there, **cellType** values 0 to 3 represent fibroblast, inflammatory, epithelial, and miscellaneous cell types respectively based on the **cellTypeName** column. In addition, **isCancerous** values 0 and 1 represent colon cells with non-Cancer and Cancer.


- The **extra_data** only contains **isCancerous (0 and 1)** column.

**=>** Therefore, we choose the **isCancerous** attribute as the target column because both **main_data** and **extra_data** have the column **isCancerous**.



- In **main_data**, only the attribute **cellType** continue to be our strongest candidate for to influence our model when it has high impact on our target attribute is **isCancerous**.


- Attributes such as **InstanceID** and **patientID** might create a lot of noise in our model. 


- There are 4 types of cells: 0, 1, 2, 3. Only cell type number 2 is cancerous while others are not cancerous.


- We expect the the model would predict class 0 more accurately than class 1

## 3.3. Data Preparation

In [60]:

traindata, validata = train_test_split(main_data[['ImageName','cellType']], 
                                              test_size=0.2, random_state=9, stratify = main_data[['cellType']])                                        

print('Training data shape task 1:', traindata.shape)
print('Validation data shape task 1:', validata.shape)
print("Sampled Dataset:\n",traindata['cellType'].value_counts())
print('Training data shape task 1:', exTrain.shape)

Training data shape task 1: (7916, 2)
Validation data shape task 1: (1980, 2)
Sampled Dataset:
 2    3263
1    2034
0    1510
3    1109
Name: cellType, dtype: int64
Training data shape task 1: (5192, 1)


# 4. Building GAN model

## 4.1 Function define

## ------------------------

### Image functions

In [61]:
#Create a a function to add image according to the name given from the list

def GetImage(directory):
    images=[]
    for name in tqdm(directory, desc="Adding images"):
        image = cv2.imread("patch_images/"+name)
        dim = (28,28)
        image = cv2.resize(image, dim)
        image = Image.fromarray(image,'RGB')
        images.append(np.array(image))
    result = np.array(images)
    print("\ngetImage COMPLETED!")
    return result



In [62]:
# Get images ready by loading and preprocessing.
def load_main_samples(traindata):
    trainX = GetImage(traindata['ImageName'])
    trainy = traindata['cellType']

    X = trainX.astype('float32')
    X = (X - 127.5) / 127.5  # scale from [0,255] to [-1,1] as we will be using tanh activation.
    return [X, trainy]


In [63]:
# Get images ready by loading and preprocessing.
def load_extra_samples(exTrain):
    trainX = GetImage(exTrain['ImageName'])
    X = trainX.astype('float32')
    X = (X - 127.5) / 127.5  # scale from [0,255] to [-1,1] as we will be using tanh activation.
    return X

In [64]:
#select subset of the dataset for supervised training
#Let us pick only 3324 samples to be used in supervised training.
#Also, we need to ensure we pick 10 samples per class to ensure a good balance
#of data between classes. 

def select_supervised_samples(dataset, n_samples=4000, n_classes=4):
    X, y = dataset
    X_list, y_list = list(), list()
    n_per_class = int(n_samples / n_classes) #Number of amples per class. 
    for i in range(n_classes):
        X_with_class = X[y == i] # get all images for this class
        ix = randint(0, len(X_with_class), n_per_class) # choose random images for each class
        [X_list.append(X_with_class[j]) for j in ix] # add to list
        [y_list.append(i) for j in ix]
    return asarray(X_list), asarray(y_list) #Returns a list of 2 numpy arrays corresponding to X and Y


## ------------------------

### Generate samples

In [65]:
#Create a function to generate sample to fix the Imblance of the dataset
def GenerateSample(X,Y):
    ros = RandomOverSampler(random_state = 1)
    x, y = ros.fit_resample(X.values.reshape(-1,1), Y)
    x = x.flatten()
    return x,y

In [66]:
# Pick real samples from the dataset.
#Return both images and corresponding labels in addition to y=1 indicating that the images are real. 
#Remember that we will not use the labels for unsupervised, only used for supervised. 
def fetch_sup_samples(dataset, n_samples):
    images, labels = dataset
    ix = randint(0, images.shape[0], n_samples)
    X, labels = images[ix], labels[ix] #Select random images and corresponding labels 
    y = ones((n_samples, 1)) #Label all images as 1 as these are real images. (for the discriminator training)
    return [X, labels], y


In [67]:
# Pick real samples from the dataset.
#Return both images and corresponding labels in addition to y=1 indicating that the images are real.
#Remember that we will not use the labels for unsupervised, only used for supervised.
def fetch_unsup_samples(dataset, n_samples):
    images = dataset
    ix = randint(0, images.shape[0], n_samples)
    X = images[ix] #Select random images and corresponding labels
    y = ones((n_samples, 1)) #Label all images as 1 as these are real images. (for the discriminator training)
    return X, y


In [68]:
# Generate fake images using the generator and above latent points as input to it. 
#We do not care about labeles so the generator will not know anything about the labels. 
def generate_fake_samples(generator, latent_dim, n_samples):
	z_input = generate_latent_points(latent_dim, n_samples)
	fake_images = generator.predict(z_input)
	# create class labels
	y = zeros((n_samples, 1)) #Label all images as 0 as these are fake images. (for the discriminator training) 
	return fake_images, y


In [69]:
# generate latent points, to be used as inputs to the generator. 
def generate_latent_points(latent_dim, n_samples):
	z_input = randn(latent_dim * n_samples)
	z_input = z_input.reshape(n_samples, latent_dim) # reshape for input to the network
	return z_input

## ------------------------

### Generator

In [70]:

n_nodes = 256 * 7 *7
X = Sequential([
    layers.Dense(n_nodes, input_dim=100),
    layers.LeakyReLU(alpha=0.2),
    layers.Reshape((7, 7, 256)),
    
    layers.Conv2DTranspose(128, (3,3), strides=(2,2), padding='same'),#14x14x128
    layers.LeakyReLU(alpha=0.2),
    
    layers.Conv2DTranspose(64, (5,5), strides=(1,1), padding='same'), #14x14x64
    layers.LeakyReLU(alpha=0.2),
    # output
    layers.Conv2DTranspose(3, (3,3), strides=(2,2), activation='tanh',padding='same')
])

opt = Adam(lr=0.0005, beta_1=0.4, beta_2=0.444, decay=1.0e-6, amsgrad=True)
X.compile(optimizer=opt , loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
X.summary()

Model: "sequential_46"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 12544)             1266944   
_________________________________________________________________
leaky_re_lu_95 (LeakyReLU)   (None, 12544)             0         
_________________________________________________________________
reshape_10 (Reshape)         (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_transpose_30 (Conv2DT (None, 14, 14, 128)       295040    
_________________________________________________________________
leaky_re_lu_96 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_transpose_31 (Conv2DT (None, 14, 14, 64)        204864    
_________________________________________________________________
leaky_re_lu_97 (LeakyReLU)   (None, 14, 14, 64)      

In [71]:
# define the standalone generator model
def define_generator(latent_dim):
    model = Sequential()
	# We will reshape input latent vector into 7x7 image as a starting point. 
    #So n_nodes for the Dense layer can be 128x8x8 so when we reshape the output 
    #it would be 8x8x128 and that can be slowly upscaled to 32x32 image for output.
    n_nodes = 256 * 7 *7  #8192 nodes
    
    model.add(layers.Dense(n_nodes, input_dim=latent_dim)) #Dense layer so we can work with 1D latent vector
    model.add( layers.LeakyReLU(alpha=0.2))
    model.add( layers.Reshape((7, 7, 256)))  #8x8x128 dataset from the latent vector.
    
    # upsample to 14x14
    model.add( layers.Conv2DTranspose(128, (3,3), strides=(1,1), padding='same')) #14x14x128
    model.add( layers.LeakyReLU(alpha=0.2))
    
    # upsample to 28x28
    model.add( layers.Conv2DTranspose(64, (5,5), strides=(2,2), padding='same')) #14x14x64
    model.add( layers.LeakyReLU(alpha=0.2))
    
    # generate
    model.add( layers.Conv2DTranspose(3, (3,3), strides=(2, 2), activation='tanh', padding='same')) #28x28x3
    assert model.output_shape == (None, 28, 28, 3)
    return model  #Model not compiled as it is not directly trained like the discriminator.
                    #Generator is trained via GAN combined model. 


test_gen = define_generator(100)
print(test_gen.summary())

## ------------------------

### Discriminator

In [72]:
# define the base discriminator model for sup and unsup discriminators
#This is the base discriminator that supervised and unsupervised are going to share weights from. 
#(I know that the above ssentence is bad english!!!)
def define_discriminator(in_shape=(28,28,3), n_classes=4):
    model = Sequential([
        layers.Conv2D(32, (3, 3),strides=(2,2), kernel_initializer='he_normal', padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(32, (3, 3),strides=(2,2), kernel_initializer='he_normal', padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.2),
        
        layers.Conv2D(64, (3, 3),kernel_initializer='he_normal', kernel_regularizer=l2(l=0.01),padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', kernel_regularizer=l2(l=0.01),padding='same'),
        layers.Dropout(0.3),
        
        layers.Conv2D(128, (3, 3),strides=(2,2), kernel_initializer='he_normal', kernel_regularizer=l2(l=0.01),padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3),strides=(2,2), kernel_initializer='he_normal', kernel_regularizer=l2(l=0.01),padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.4),
        
        layers.Flatten(),
        layers.Dense(512, kernel_regularizer=l2(l=0.01)),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.2),
        layers.Dense(512, kernel_regularizer=l2(l=0.01)),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.2),
        layers.Dense(n_classes)
    ])
    return model

In [73]:
#Define the supervised discriminator. 
#Multiclass classification, so we will use softmax activation.
#To avoid converting our labels to categorical, we will work with sparse categorical crossentropy loss. 
def define_sup_discriminator(disc):
    model=Sequential()
    model.add(disc)
    model.add(layers.Activation('softmax'))
    #Let us use sparse categorical loss so we dont have to convert our Y to categorical
    model.compile(optimizer=Adam(lr=0.0002, beta_1=0.5), 
                  loss="sparse_categorical_crossentropy",metrics=['accuracy'])
    
    return model

In [74]:
# custom activation function for the unsupervised discriminator
#D(x) = Z(x) / (Z(x) + 1) where Z(x) = sum(exp(l(x))). l(x) is the output from sup discr. prior to softmax
def custom_activation(x):
    Z_x = K.sum(K.exp(x), axis=-1, keepdims=True)
    D_x = Z_x /(Z_x+1)
    
    return D_x

#You can also try the built in tensorflow function: tf.reduce_logsumexp(class_logits, 1)

def define_unsup_discriminator(disc):
    model=Sequential()
    model.add(disc)
    model.add(layers.Lambda(custom_activation))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
    return model

## ------------------------

### GAN

In [75]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(gen_model, disc_unsup):
    disc_unsup.trainable = False # make unsup. discriminator not trainable
    model = Sequential()
    model.add(gen_model)
    model.add(disc_unsup)
    opt = Adam(lr=0.0002, beta_1=0.5, beta_2=0.444, decay=1.0e-8, amsgrad=True)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model


In [76]:
import math
# train the generator and discriminator
def train(gen_model, disc_unsup, disc_sup, gan_model, dataset,extraData, latent_dim, n_epochs=20, n_batch=100):

    # select supervised dataset for training.
    #Remember that we are not using all 60k images, just a subset (100 images, 10 per class. )
    X_sup, y_sup = select_supervised_samples(dataset)
    # 	print(X_sup.shape, y_sup.shape)

    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    # iterations
    n_steps = bat_per_epo * n_epochs

    half_batch = int(n_batch / 2)
#     print('n_epochs=%d, n_batch=%d, 1/2=%d, b/e=%d, steps=%d' % (n_epochs, 
#                                                           n_batch, half_batch, 
#                                                           bat_per_epo, n_steps))
    #  enumerate epochs
    for i in range(n_steps):
        # update supervised discriminator (disc_sup) on real samples.
        #Remember that we use real labels to train as this is supervised. 
        #This is the discriminator we really care about at the end.
        #Also, this is a multiclass classifier, not binary. Therefore, our y values 
        #will be the real class labels for MNIST. (NOT 1 or 0 indicating real or fake.)
        [Xsup_real, ysup_real], _ = fetch_sup_samples([X_sup, y_sup], half_batch)

        sup_loss, sup_acc = disc_sup.train_on_batch(Xsup_real, ysup_real)

        # update unsupervised discriminator (disc_unsup) - just like in our regular GAN.
        #Remember that we will not train on labels as this is unsupervised, just binary as in our regular GAN.
        #The y_real below indicates 1s telling the discriminator that these images are real. 
        #do not confuse this with class labels. 
        #We will discard this discriminator at the end. 

        X_real, y_real = fetch_unsup_samples(dataset[0], half_batch) #
        d_loss_real = disc_unsup.train_on_batch(X_real, y_real)

        #Now train on fake. 
        X_fake, y_fake = generate_fake_samples(gen_model, latent_dim, half_batch)
        d_loss_fake = disc_unsup.train_on_batch(X_fake, y_fake)

        # update generator (gen) - like we do in regular GAN.
        #We can discard this model at the end as our primary goal is to train a multiclass classifier (sup. disc.)
        X_gan, y_gan = generate_latent_points(latent_dim, n_batch), ones((n_batch, 1))
        gan_loss = gan_model.train_on_batch(X_gan, y_gan)

        # summarize loss on this batch
        print('>%d, c[%.3f,%.0f], d[%.3f,%.3f], g[%.3f]' % (i+1, sup_loss, sup_acc*100, d_loss_real, d_loss_fake, gan_loss))
        if(math.isnan(gan_loss)):
            break;
        # evaluate the model performance periodically
        if (i+1) % (bat_per_epo * 1) == 0:
            summarize_performance(i, gen_model, disc_sup, latent_dim, dataset)


In [77]:
# report accuracy and save plots & the model periodically.
def summarize_performance(step, gen_model, disc_sup, latent_dim, dataset, n_samples=100):
    # Generate fake images
    X, _ = generate_fake_samples(gen_model, latent_dim, n_samples)

    X = (X + 1) / 2.0 # scale to [0,1] for plotting
    # plot images
    for i in range(12):
        plt.subplot(4, 4, 1 + i)
        plt.axis('off')
        plt.imshow(X[i, :, :, 0], cmap='gray_r')
    # save plot to drive
    filename1 = 'generated_plot_%04d.png' % (step+1)
    plt.savefig('GAN/gen_pic/'+filename1)
    plt.close()

    # evaluate the discriminator
    X, y = dataset
    _, acc = disc_sup.evaluate(X, y, verbose=0)
    print('Discriminator Accuracy: %.3f%%' % (acc * 100))
    # save the generator model
    filename2 = 'gen_model_%04d.h5' % (step+1)
    gen_model.save('GAN/generator/'+filename2)
    
    # save the Discriminator (classifier) model
    filename3 = 'disc_sup_%04d.h5' % (step+1)
    disc_sup.save('GAN/classifier/'+filename3)
    print('>Saved: %s, %s, and %s' % (filename1, filename2, filename3))

## ------------------------

## 4.2 Model training

In [78]:
latent_dim = 100

# create the discriminator models
disc=define_discriminator() #Bare discriminator model... 
disc_sup=define_sup_discriminator(disc) #Supervised discriminator model
disc_unsup=define_unsup_discriminator(disc) #Unsupervised discriminator model. 

gen_model = define_generator(latent_dim) #Generator
gan_model = define_gan(gen_model, disc_unsup) #GAN
dataset = load_main_samples(traindata) #Define the dataset by loading real samples. (This will be a list of 2 numpy arrays, X and y)
extraData = load_extra_samples(exTrain) #Define the dataset by loading real samples. (This will be a list of 2 numpy arrays, X and y)
# train the model
# NOTE: 1 epoch = 600 steps in this example. 
with tf.device('/GPU:0'): 
    train(gen_model, disc_unsup, disc_sup, gan_model, dataset,extraData, latent_dim, n_epochs=100, n_batch=32)

Adding images: 100%|█████████████████████████████████████████████████████████████| 7916/7916 [00:01<00:00, 7422.05it/s]



getImage COMPLETED!


Adding images: 100%|█████████████████████████████████████████████████████████████| 5192/5192 [00:00<00:00, 7423.99it/s]



getImage COMPLETED!
>1, c[20.038,6], d[18.071,19.448], g[17.931]
>2, c[19.356,19], d[17.800,19.236], g[17.694]
>3, c[19.584,0], d[17.569,19.005], g[17.457]
>4, c[18.992,25], d[17.343,18.792], g[17.229]
>5, c[18.833,19], d[17.096,18.550], g[17.009]
>6, c[18.252,25], d[16.867,18.313], g[16.802]
>7, c[18.025,25], d[16.666,18.044], g[16.595]
>8, c[17.973,6], d[16.428,17.781], g[16.394]
>9, c[17.623,31], d[16.218,17.569], g[16.191]
>10, c[17.637,25], d[16.016,17.342], g[15.983]
>11, c[17.469,25], d[15.815,17.150], g[15.779]
>12, c[17.015,31], d[15.575,16.976], g[15.572]
>13, c[16.811,38], d[15.368,16.776], g[15.369]
>14, c[16.884,12], d[15.201,16.574], g[15.168]
>15, c[16.742,19], d[14.991,16.381], g[14.973]
>16, c[16.291,31], d[14.802,16.173], g[14.780]
>17, c[16.242,19], d[14.594,15.972], g[14.592]
>18, c[15.970,19], d[14.412,15.767], g[14.410]
>19, c[15.481,31], d[14.209,15.608], g[14.221]
>20, c[15.340,31], d[14.008,15.439], g[14.032]
>21, c[15.393,12], d[13.833,15.262], g[13.852]
>22,

>186, c[4.297,25], d[2.949,2.947], g[8.214]
>187, c[4.316,31], d[2.926,2.935], g[7.361]
>188, c[4.105,56], d[2.935,2.918], g[7.917]
>189, c[4.086,38], d[2.891,2.891], g[7.630]
>190, c[4.040,38], d[2.883,2.878], g[8.174]
>191, c[4.102,38], d[2.965,2.949], g[7.840]
>192, c[3.945,44], d[2.866,2.835], g[8.032]
>193, c[4.099,25], d[2.855,2.824], g[7.358]
>194, c[3.960,44], d[2.808,2.888], g[8.761]
>195, c[4.030,44], d[2.863,2.780], g[8.852]
>196, c[3.868,62], d[2.957,2.914], g[7.475]
>197, c[4.090,38], d[2.775,2.756], g[7.745]
>198, c[4.019,31], d[2.751,2.737], g[7.464]
>199, c[3.918,38], d[2.767,2.738], g[6.919]
>200, c[3.751,56], d[2.711,2.727], g[6.523]
>201, c[3.947,19], d[2.739,2.714], g[6.416]
>202, c[3.812,50], d[2.710,2.697], g[6.418]
>203, c[3.888,38], d[2.667,2.711], g[6.371]
>204, c[3.914,31], d[2.645,2.671], g[6.916]
>205, c[3.944,31], d[2.630,2.644], g[6.654]
>206, c[3.656,50], d[2.615,2.622], g[7.028]
>207, c[4.110,25], d[2.596,2.632], g[6.017]
>208, c[3.971,25], d[2.578,2.622

...

>23892, c[0.817,75], d[1.036,0.770], g[1.144]
>23893, c[1.442,50], d[0.947,0.929], g[1.234]
>23894, c[0.890,75], d[0.652,1.018], g[1.163]
>23895, c[0.636,88], d[0.754,0.995], g[1.204]
>23896, c[1.326,56], d[0.836,0.766], g[1.114]
>23897, c[0.918,69], d[0.713,0.956], g[1.161]
>23898, c[0.713,88], d[0.830,0.860], g[1.292]
>23899, c[0.910,69], d[0.751,0.723], g[1.318]
>23900, c[0.820,69], d[0.887,0.810], g[1.363]
>23901, c[1.453,38], d[0.770,0.842], g[1.293]
>23902, c[1.078,62], d[0.720,0.899], g[1.342]
>23903, c[0.761,81], d[0.905,0.826], g[1.243]
>23904, c[1.167,44], d[0.933,0.862], g[1.296]
>23905, c[1.001,69], d[0.692,0.787], g[1.238]
>23906, c[1.261,62], d[0.781,0.842], g[1.228]
>23907, c[0.861,69], d[0.874,0.894], g[1.235]
>23908, c[1.083,50], d[0.726,0.737], g[1.219]
>23909, c[0.928,62], d[0.899,0.810], g[1.255]
>23910, c[0.964,75], d[0.799,0.744], g[1.240]
>23911, c[1.261,62], d[0.977,0.932], g[1.278]
>23912, c[0.966,62], d[0.749,0.907], g[1.217]
>23913, c[0.839,62], d[0.663,0.931

>24065, c[0.771,81], d[0.961,0.868], g[1.335]
>24066, c[1.203,62], d[1.010,0.846], g[1.287]
>24067, c[0.952,75], d[0.918,0.724], g[1.218]
>24068, c[0.981,56], d[0.843,0.720], g[1.149]
>24069, c[0.821,75], d[0.789,1.060], g[1.150]
>24070, c[1.131,69], d[0.738,0.847], g[1.198]
>24071, c[0.893,69], d[0.821,0.872], g[1.208]
>24072, c[0.786,75], d[0.761,0.679], g[1.130]
>24073, c[1.394,44], d[0.994,0.732], g[1.115]
>24074, c[0.741,81], d[0.797,0.776], g[1.127]
>24075, c[0.985,81], d[0.904,0.898], g[1.240]
>24076, c[0.883,75], d[0.876,0.961], g[1.213]
>24077, c[1.441,56], d[0.748,0.886], g[1.079]
>24078, c[1.128,69], d[0.755,0.944], g[1.065]
>24079, c[1.043,69], d[0.682,0.795], g[1.217]
>24080, c[0.754,81], d[0.799,0.723], g[1.092]
>24081, c[1.269,69], d[0.698,1.070], g[1.080]
>24082, c[1.015,62], d[0.795,0.977], g[1.205]
>24083, c[1.110,69], d[0.847,0.840], g[1.239]
>24084, c[0.885,69], d[0.913,0.754], g[1.287]
>24085, c[0.777,88], d[0.994,0.711], g[1.198]
>24086, c[0.907,75], d[0.869,0.791

>24240, c[0.836,69], d[0.742,0.818], g[1.221]
>24241, c[0.959,75], d[1.021,0.797], g[1.213]
>24242, c[0.849,75], d[0.879,0.886], g[1.252]
>24243, c[1.288,62], d[0.980,0.806], g[1.081]
>24244, c[0.835,75], d[0.846,0.801], g[1.171]
>24245, c[0.838,75], d[0.897,0.896], g[1.226]
>24246, c[0.804,81], d[0.821,0.939], g[1.262]
>24247, c[0.787,81], d[0.801,0.650], g[1.253]
>24248, c[1.051,75], d[0.698,0.952], g[1.253]
>24249, c[0.976,56], d[0.878,0.750], g[1.276]
>24250, c[0.980,69], d[0.917,0.649], g[1.337]
>24251, c[0.644,94], d[0.789,0.727], g[1.250]
>24252, c[0.904,69], d[0.873,0.930], g[1.148]
>24253, c[0.740,81], d[0.692,0.984], g[1.251]
>24254, c[0.733,81], d[0.797,0.698], g[1.309]
>24255, c[1.056,62], d[0.794,0.756], g[1.426]
>24256, c[0.765,81], d[0.818,0.723], g[1.306]
>24257, c[1.173,56], d[0.830,0.850], g[1.270]
>24258, c[1.089,62], d[0.922,0.765], g[1.346]
>24259, c[1.287,50], d[0.892,0.983], g[1.044]
>24260, c[0.888,69], d[0.978,0.890], g[1.137]
>24261, c[0.907,69], d[0.974,0.795

>24420, c[0.964,62], d[0.559,0.918], g[1.273]
>24421, c[0.858,81], d[0.831,0.796], g[1.244]
>24422, c[0.836,81], d[0.829,0.724], g[1.261]
>24423, c[0.706,75], d[0.924,0.825], g[1.272]
>24424, c[0.795,75], d[0.914,0.979], g[1.187]
>24425, c[0.703,81], d[0.795,0.996], g[1.166]
>24426, c[1.047,62], d[0.958,0.924], g[1.258]
>24427, c[1.178,56], d[0.886,0.837], g[1.169]
>24428, c[0.927,75], d[0.900,0.885], g[1.101]
>24429, c[1.256,50], d[0.890,0.998], g[1.102]
>24430, c[0.973,62], d[0.899,0.892], g[1.114]
>24431, c[0.789,75], d[0.925,0.963], g[1.182]
>24432, c[0.916,69], d[0.969,0.743], g[1.194]
>24433, c[1.124,56], d[0.975,0.794], g[1.184]
>24434, c[0.852,81], d[0.835,0.736], g[1.076]
>24435, c[1.171,75], d[0.746,1.009], g[1.102]
>24436, c[0.580,88], d[0.739,0.848], g[1.336]
>24437, c[1.022,75], d[0.730,0.651], g[1.359]
>24438, c[0.814,81], d[0.974,0.723], g[1.348]
>24439, c[1.145,62], d[1.057,0.594], g[1.101]
>24440, c[0.942,69], d[0.842,0.749], g[1.240]
>24441, c[0.804,88], d[0.862,0.839

>24594, c[0.979,62], d[0.731,0.776], g[1.312]
>24595, c[0.801,81], d[1.012,0.795], g[1.256]
>24596, c[0.954,62], d[0.819,0.694], g[1.382]
>24597, c[0.614,94], d[0.961,0.630], g[1.304]
>24598, c[0.918,62], d[0.932,0.855], g[1.253]
>24599, c[0.867,81], d[0.978,0.753], g[1.299]
>24600, c[1.006,75], d[0.812,0.789], g[1.179]
>24601, c[0.773,88], d[0.809,0.882], g[1.338]
>24602, c[0.964,69], d[0.994,0.816], g[1.145]
>24603, c[1.132,44], d[0.829,0.773], g[1.202]
>24604, c[1.057,62], d[0.846,0.730], g[1.264]
>24605, c[0.633,94], d[0.922,0.913], g[1.249]
>24606, c[0.966,81], d[0.793,0.906], g[1.239]
>24607, c[0.614,94], d[0.990,0.863], g[1.107]
>24608, c[0.860,75], d[0.856,0.895], g[1.236]
>24609, c[0.743,81], d[0.697,0.906], g[1.086]
>24610, c[1.157,62], d[0.719,0.910], g[1.177]
>24611, c[0.933,75], d[0.818,1.001], g[1.141]
>24612, c[0.774,81], d[0.785,0.822], g[1.246]
>24613, c[0.808,69], d[0.736,0.816], g[1.252]
>24614, c[0.905,69], d[0.813,0.839], g[1.243]
>24615, c[0.911,75], d[0.910,0.843

## 4.3 Prediction

In [98]:
from tensorflow.keras.models import load_model
classifier= load_model("GAN\classifier\disc_sup_24700.h5")

In [99]:
from sklearn.metrics import classification_report

valX = GetImage(validata['ImageName'])
valY = validata['cellType']

valX = valX.astype('float32')
valX = (valX - 127.5) / 127.5  # scale from [0,255] to [-1,1] as we will be using tanh activation.

predict=model.predict(valX)
classes=np.argmax(predict,axis=1)
print(classification_report(valY, classes))

Adding images: 100%|█████████████████████████████████████████████████████████████| 1980/1980 [00:00<00:00, 7446.06it/s]



getImage COMPLETED!
              precision    recall  f1-score   support

           0       0.12      0.04      0.06       378
           1       0.00      0.00      0.00       509
           2       0.40      0.92      0.56       816
           3       0.00      0.00      0.00       277

    accuracy                           0.39      1980
   macro avg       0.13      0.24      0.15      1980
weighted avg       0.19      0.39      0.24      1980



In [80]:
# class_names = ["fibroblast","inflammatory","epithelial","others"]
# extraX = GetImage(extra_data['ImageName'])
# extraY = extra_data['isCancerous']

# extraX = extraX.astype('float32')
# extraY = (extraY - 127.5) / 127.5  # scale from [0,255] to [-1,1] as we will be using tanh activation.

# predict=model.predict(extraX)
# classes=np.argmax(predict,axis=1)
# names = list()

# for i in range(len(classes)):
#     names.append(class_names[np.argmax(predict[i])])
    
# submit = pd.DataFrame(columns = ['InstanceID',  'patientID',  'ImageName','cellTypeName','cellType','isCancerous',])
# submit['InstanceID'] = extra_data['InstanceID']
# submit['patientID'] = extra_data['patientID']
# submit['ImageName'] = extra_data['ImageName']
# submit['isCancerous'] = extra_data['isCancerous']
# submit['cellType'] = classes
# submit['cellTypeName'] = names
# submit.to_csv("labeled_extra.csv", index = False)